### **🔍 ¿Qué es TF-IDF?**
El `TF-IDF` es una técnica usada en NLP (Procesamiento de Lenguaje Natural) que convierte texto en una **matriz numérica** e **indica qué tan importante es una palabra dentro de un documento** comparado con todos los documentos del corpus.

![Descripción opcional](http://www.mblazquez.es/blog-ccdoc-recuperacion/formulas/formula05_ponderacion-tf-idf.png)
<!--  -->
TF-IDF significa:
- **TF (Term Frequency):** cuántas veces aparece una palabra en un documento.
- **IDF (Inverse Document Frequency):** mide qué tan rara o informativa es una palabra en el corpus.

#### **🧠 Fórmulas**
**TF(t, d)** = (Frecuencia del término t en el documento d) / (Total de terminos en d)
<!--  -->
**IDF(t)** = log(Nº total de documentos / Nº de documentos que contienen el término t)

Entonces:

**TF-IDF(t, d)** = TF(t, d) × IDF(t)

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
corpus = [
    'el gato negro',
    'el gato blanco',
    'el perro negro'
]

# Stopwords -> Palabras muy comunes a eliminar para reducir ruido 
stopwords_es = {'el', 'en', 'la'}

# Limpiar documentos (eliminar stopwords)
tokenized_docs = []
for doc in corpus:
    words = doc.lower().split()
    filtered = [word for word in words if word not in stopwords_es]
    tokenized_docs.append(filtered)
    print(filtered)

# Crear vocabulario (palabras únicas)
vocab = sorted(set(word for doc in tokenized_docs for word in doc))
print()
print(vocab)
print()
print(tokenized_docs)

['gato', 'negro']
['gato', 'blanco']
['perro', 'negro']

['blanco', 'gato', 'negro', 'perro']

[['gato', 'negro'], ['gato', 'blanco'], ['perro', 'negro']]


In [3]:
# IDF(t) = log(Nº total de documentos / Nº de documentos que contienen el término t)
def compute_idf(tokenized_docs):
    idf = {}
    N = len(tokenized_docs)
    for term in vocab:
        containing_docs = sum(1 for doc in tokenized_docs if term in doc)
        idf[term] = math.log((1+N) / (1+containing_docs)) + 1  # +1 para evitar división por 0
    return idf

idf = compute_idf(tokenized_docs)
idf

{'blanco': 1.6931471805599454,
 'gato': 1.2876820724517808,
 'negro': 1.2876820724517808,
 'perro': 1.6931471805599454}

In [4]:
# TF(t, d) = (Nº de veces que el término t aparece en el documento d)
def compute_tf(doc_tokens):
    tf = {}
    total_terms = len(doc_tokens)
    for term in vocab:
        tf[term] = doc_tokens.count(term) / total_terms
    return tf

# Calcular TF-IDF por documento
tf_dict = []

for doc in tokenized_docs:
    tf_doc = compute_tf(doc)
    tf_dict.append(tf_doc)

tf_dict

[{'blanco': 0.0, 'gato': 0.5, 'negro': 0.5, 'perro': 0.0},
 {'blanco': 0.5, 'gato': 0.5, 'negro': 0.0, 'perro': 0.0},
 {'blanco': 0.0, 'gato': 0.0, 'negro': 0.5, 'perro': 0.5}]

In [8]:
def l2_normalize(vector):
    norm = math.sqrt(sum(v**2 for v in vector.values()))
    if norm == 0:
        return vector
    return {k: v / norm for k, v in vector.items()}

# Calcular TF-IDF por documento
tfidf_vectors_dict = []
tfidf_vectors = []

for i, doc in enumerate(tokenized_docs):
    tfidf = {term: tf_dict[i][term] * idf[term] for term in vocab}
    tfidf = l2_normalize(tfidf)

    tfidf_vectors_dict.append(tfidf)
    tfidf_vectors.append(list(tfidf.values()))

print('VOCABULARIO:', vocab)

print('\n\nTF-IDF')
for i, vec in enumerate(tfidf_vectors_dict):
    print(f'\nDocumento {i+1}')
    for term in vocab:
        print(f'{term:10}: {vec[term]:.3f}')

np.array(tfidf_vectors)

VOCABULARIO: ['blanco', 'gato', 'negro', 'perro']


TF-IDF

Documento 1
blanco    : 0.000
gato      : 0.707
negro     : 0.707
perro     : 0.000

Documento 2
blanco    : 0.796
gato      : 0.605
negro     : 0.000
perro     : 0.000

Documento 3
blanco    : 0.000
gato      : 0.000
negro     : 0.605
perro     : 0.796


array([[0.        , 0.70710678, 0.70710678, 0.        ],
       [0.79596054, 0.60534851, 0.        , 0.        ],
       [0.        , 0.        , 0.60534851, 0.79596054]])

## **TfidfVectorizer**

In [6]:
# Stopwords
spanish_stopwords = stopwords.words('spanish')

vectorizer = TfidfVectorizer(stop_words=spanish_stopwords,
                             norm='l2',
                             use_idf=True,
                             smooth_idf=True,
                             sublinear_tf=False)
X = vectorizer.fit_transform(corpus)

print('VOCABULARIO:', vectorizer.get_feature_names_out())
print()
print(X.toarray())

VOCABULARIO: ['blanco' 'gato' 'negro' 'perro']

[[0.         0.70710678 0.70710678 0.        ]
 [0.79596054 0.60534851 0.         0.        ]
 [0.         0.         0.60534851 0.79596054]]
